<a href="https://colab.research.google.com/github/vaishnavireddya11/Assignment-03-/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 1st Question ###

In [3]:
from transformers import pipeline
from huggingface_hub import InferenceClient
import traceback

In [4]:
try:
  local_generator = pipeline("text-generation", model="gpt2")
except Exception as e:
  print("Failed to load GTP-2 locally:", e)
  local_generator = None

Device set to use cpu


In [5]:
HF_API_TOKEN = ""
client = InferenceClient(
    model = "mistralai/Mistral-7B-Instruct-v0.2",
    token = HF_API_TOKEN
)

In [ ]:
def generate_text(prompt):
  try:
    print("Trying GPT-2 locally...")
    if local_generator:
      result = local_generator(prompt, max_length = 50)[0]['generated_text']
      return f"GPT-2 (Local):\n{result}"
    else:
      raise RuntimeError("GPT-2 not available")
  except Exception as e:
    print("Local GPT-2 failed. Falling back to Hugging Face Inference API...")
    try:
      messages = [
          {"role": "system", "content":"You are a helpful assistant."},
          {"role": "user","content": prompt}
      ]
      response = client.chat_completion(messages=messages, max_tokens = 50)
      result = response.choices[0].message.content.strip()
      return f"Hugging Face API: \n{result}"
    except Exception as api_error:
      return f"Both methods failed. \nError:\n{traceback.format_exc()}"
user_input = input("Enter your prompt: ")
output = generate_text(user_input)
print("\n" + output)

In [ ]:
### 2nd Question ###

In [2]:
# Install required library if not already installed:
# pip install transformers

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load pre-trained model and tokenizer (GPT-2)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Set pad token to avoid warnings
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

def generate_story(prompt, temperature=1.0):
    """
    Generates a story based on the input prompt and temperature.
    Uses sampling methods like top_k, top_p and repetition penalty.
    Includes exception handling for robustness.
    """
    try:
        # Tokenize input prompt and convert to tensor
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        # Generate output
        output = model.generate(
            input_ids,
            max_length=150,             # Limit max length
            do_sample=True,             # Use sampling
            temperature=temperature,    # Control randomness
            top_k=50,                   # Limit to top_k most likely words
            top_p=0.95,                 # Nucleus sampling
            repetition_penalty=1.2,     # Penalize repetition
            pad_token_id=tokenizer.eos_token_id # Avoid warnings
        )

        # Decode and print generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f"\n Temperature: {temperature}\n{generated_text}\n{'-'*50}")
        return generated_text

    except Exception as e:
        print(f"Error during generation: {str(e)}")
        return None


# --------------------------
# Test Cases with Prompt
# --------------------------

prompt = "Once upon a time in a distant galaxy, a young explorer discovered"

# Test with various temperature values to observe differences
temperatures = [0.3, 0.7, 1.0, 1.3]

for temp in temperatures:
    generate_story(prompt, temperature=temp)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



 Temperature: 0.3
Once upon a time in a distant galaxy, a young explorer discovered the world of K'un-Tai. The planet was inhabited by an ancient race known as Yggdrasil and they were called to investigate it for their own purposes.[1]
 (ENT: "The Last Jedi")
--------------------------------------------------

 Temperature: 0.7
Once upon a time in a distant galaxy, a young explorer discovered that the planet Pluto had been made uninhabitable by an asteroid. He then took his discovery to Earth and built another telescope on it where he could check for life while still traveling at full speed through space...and found out there were indeed some people who lived within this world!
You've probably already seen what I'm talking about; if not you're going to be missing something very important here: An entire chapter of "The Odyssey" is dedicated to exploring exactly how these two worlds came into being as they did…but before we get started with their history or any further information (not